In [1]:
import numpy as np
import random
import pandas as pd 
from copy import deepcopy
import math
import time
from math import ceil,floor

In [2]:
def g1(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    
    return (-x1 + 0.0193*x3)

In [3]:
def g2(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
   
    
    return (-x2 + 0.0095*x3)

In [4]:
def g3(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    pi=math.pi

    return  (-pi*x3**2*x4 - (4/3)*pi*x3**3 + 1296000) 

In [5]:
def g4(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    return (x4-240)

In [6]:
def is_feasable(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
  
    if x1<0.0625  or x1>99*0.0625:
        #print("x1")
        return False
    if x2<0.0625 or x2>99*0.0625:
        #print("x2")
        return False
        
    if x3<10 or x3>200:
        #print("x3")
        return False
    if x4<10 or x4>200:
        #print("x4")
        return False
    

    g1res=g1(x)
    g2res=g2(x)
    g3res=g3(x)
    g4res=g4(x)
    
    if  g1res>0:
        #print("g1")
        return False
    if g2res>0:
        #print("g2")
        return False
    if  g3res>0:
        #print("g3")
        return False
    if g4res>0:
        return False
    return True

In [7]:
def vessel(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    
    toReturn= -(0.6224*x1*x3*x4 + 1.7781*x2*x3**2 + 3.1661*x1**2*x4 + 19.84*x1**2*x3)
    penalty=50000
    g1res=g1(x)
    if g1res>0:
        toReturn+=-g1res*penalty
    
    
    g2res=g2(x)
    if g2res>0:
        toReturn+=-g2res*penalty
    
    
    g3res=g3(x)
    if g3res>0:
        toReturn+=-g3res*penalty
    g4res=g4(x)
    if g4res>0:
        toReturn+=-g4res*penalty
    return toReturn

In [8]:
def actual_vessel(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    
    
    return -(0.6224*x1*x3*x4 + 1.7781*x2*x3**2 + 3.1661*x1**2*x4 + 19.84*x1**2*x3)

In [9]:
class Individual:
    global_best_position=None
    global_best_fitness=None
    def __init__(self,bounds, velocity_calc,calc_fitness):
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])        
        
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        self.velocity = np.random.uniform(self.lower_bounds - self.upper_bounds,
                                          self.upper_bounds - self.lower_bounds,
                                          len(bounds))
        self.calc_fitness=calc_fitness
        self.fitness=calc_fitness(self.position)
        

        self.velocity_calc=velocity_calc
        
        self.personal_best_fitness=self.fitness
        self.personal_best_position=self.position.copy()
        
        if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
    
        
    def update_position(self):
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)
        self.update_fitness()
       
                
    def update_velocity(self):
        personal_direction=(self.personal_best_position - self.position)
        global_direction=(Individual.global_best_position - self.position)
                
        cognitive_velocity =random.random() * self.velocity_calc["cognitive"]*personal_direction
        social_velocity =random.random() * self.velocity_calc["group"]* global_direction
        
        inertia = self.velocity_calc["inertia"] * self.velocity
        self.velocity = inertia + cognitive_velocity + social_velocity
        self.velocity=np.clip(self.velocity,2*self.lower_bounds,2*self.upper_bounds)
    def update_fitness(self):
        self.fitness=self.calc_fitness(self.position)
        if self.personal_best_fitness is None or self.fitness>self.personal_best_fitness:
            self.personal_best_fitness=self.fitness
            self.personal_best_position=self.position.copy()
            if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
    

In [10]:
def selection(population:Individual) ->Individual:
    #roulete
    ranks= list.reverse(list(range(1,len(population)+1)))
    return random.choices(population=list(zip(range(len(population)),population)),weights=ranks,k=1)[0]

In [11]:
def crossover(p1:Individual,p2:Individual,c1:Individual,c2:Individual):
    coef=random.random()
    c1.position=coef*p1.position + (1-coef)*p2.position

    c2.position=coef*p2.position +(1-coef)*p1.position
    

In [12]:
def mutation(ind:Individual,mut_prob=0.01):
    #ind.mutate(mut_prob)
    pass

In [13]:
def GA(ind:Individual,pop_size,num_iters:int,elitism_size=6):
    bounds=list(zip(ind.lower_bounds,ind.upper_bounds))
    population=[Individual(bounds,ind.velocity_calc,ind.calc_fitness) for _ in range(pop_size)]
    population[0]=ind
    if pop_size%2==1:
        pop_size-=1
    new_population=population[:]
    for _ in range(0,num_iters):
        sorted(population,key=lambda x:x.fitness,reverse=True)
        for i in range(elitism_size,pop_size,2):
            p1=selection(population)[1]
            p2=selection(population)[1]
            
            crossover(p1,p2,new_population[i],new_population[i+1])
            mutation(new_population[i])
            mutation(new_population[i+1])
            new_population[i].update_fitness()
            new_population[i+1].update_fitness()
        population=new_population[:]
    return max(population,key=lambda x:x.fitness)

In [14]:
vezbe_velocity_calc={
    "cognitive":0.7 ,
    "group":1.0 ,
    "inertia":1.0
    
}

In [15]:
def hybrid(swarm_size,num_dimensions,num_iters,allowed_time,velocity_calc,fitness_calc,ga_min_num,ga_max_num,ga_min_iter,ga_max_iter,ga_min_ps,ga_max_ps,y,b):
    ga_num=ga_max_num
    ga_pop_size=ga_min_ps
    ga_iters=ga_min_iter
    bounds = [(0.0625,99* 0.0625),(0.0625,99* 0.0625),(10,200),(10,200)]
    
    swarm = [Individual(bounds=bounds,
                      calc_fitness=fitness_calc,velocity_calc=velocity_calc,
                      ) for i in range(swarm_size)]
    #print(Individual.global_best_fitness)
    
    
    start=time.time()
    
    for psoiter in range(num_iters):
        #sorted(population,key=lambda x:-x.fitness)
        #pso
        for i in range(len(swarm)):
               
            swarm[i].update_velocity()
            swarm[i].update_position()
            
            swarm[i].update_fitness()
        if time.time()-start>allowed_time:
            tmp=Individual.global_best_position
                #if not is_feasable(Individual.global_best_position):
                    #print("not feasable!")
                    #print(g1(Individual.global_best_position))
                    #print(g3(Individual.global_best_position))
            Individual.global_best_fitness=None
            Individual.global_best_position=None
            #print("actual time: "+str(time.time()-start))
            return tmp
        #GAs    
        for i in range(ga_num):
            tmp=selection(swarm)
            index=tmp[0]
            selected=tmp[1]
            
            swarm[index]=GA(selected,ga_pop_size,ga_iters)
        #update
        
        ga_num=int(ga_max_num-(psoiter/num_iters)**y*(ga_max_num-ga_min_num))
        ga_pop_size=int(ga_min_ps+(psoiter/num_iters)**y*(ga_max_ps-ga_min_ps))
        ga_iters=int(ga_min_iter+(psoiter/num_iters)**b*(ga_max_iter-ga_min_iter))
        
        if time.time()-start>allowed_time:
                tmp=Individual.global_best_position
                #if not is_feasable(Individual.global_best_position):
                    #print("not feasable!")
                    #print(g1(Individual.global_best_position))
                    #print(g3(Individual.global_best_position))
                Individual.global_best_fitness=None
                Individual.global_best_position=None
                #print("actual time: "+str(time.time()-start))
                return tmp
    print("got out")
    return max(swarm,key=lambda x:x.fitness)

In [16]:
tmp=hybrid(swarm_size=50,num_dimensions=5,num_iters=100,allowed_time=0.1,velocity_calc=vezbe_velocity_calc,fitness_calc=
       himmelblau,ga_min_num=10,ga_max_num=50,ga_min_ps=50,ga_max_ps=100,ga_min_iter=20,ga_max_iter=100,y=2,b=3)
if not is_feasable(tmp):
    print("not feasable!")
print(actual_himmelblau(tmp))

NameError: name 'himmelblau' is not defined

In [ ]:
times=[5*10**(-2),10**(-1),1]
num_trys=1000
print("Function : Himbelblau")
print("Swarm size: " + "50")
print("Num dimensions: " + "5")
print("c_i: "+"0.7")
print("c_p: " + "1.0")
print("c_g: " + "1.0")
print("ga_min_num : 10")
print("ga_max_num :50")
print("ga_min_ps : 50")
print("ga_max_ps : 100")
print("ga_min_iter : 20")
print("ga_max_iter : 100")
print("y : 2")
print("b :3")
for allowed_time in times:
    suma=0
    counter=0
    for _ in range(num_trys):
        position=hybrid(swarm_size=50,num_dimensions=5,num_iters=100,allowed_time=allowed_time,velocity_calc=vezbe_velocity_calc,fitness_calc=
        vessel,ga_min_num=10,ga_max_num=30,ga_min_ps=20,ga_max_ps=40,ga_min_iter=20,ga_max_iter=40,y=2,b=3)
        if is_feasable(position):
            suma+=actual_vessel(position)
            counter+=1
    suma=-suma
    suma/=counter
    
    print("    Time : " + str(allowed_time))
    print("    NumHits : "+ str(counter) + "/" + str(num_trys))
    print("    Average minimum: " +str(suma))
    print("\n")

In [ ]:
#hybrid(swarm_size=50,num_dimensions=2,num_iters=100,allowed_time=1,velocity_calc=vezbe_velocity_calc,fitness_calc=shaffer
 #      ,ga_min_num=10,ga_max_num=50,ga_min_ps=50,ga_max_ps=100,ga_min_iter=20,ga_max_iter=100,y=2,b=3)

In [17]:
suma=0
counter=0
for _ in range(50):
    position=hybrid(swarm_size=50,num_dimensions=5,num_iters=5000,allowed_time=50,velocity_calc=vezbe_velocity_calc,fitness_calc=
    vessel,ga_min_num=10,ga_max_num=30,ga_min_ps=20,ga_max_ps=40,ga_min_iter=20,ga_max_iter=40,y=2,b=3)
    if is_feasable(position):
        suma+=actual_vessel(position)
        counter+=1
suma=-suma
suma/=counter
    
print("    Time : " + str(50))
print("    NumHits : "+ str(counter) + "/" + str(50))
print("    Average minimum: " +str(suma))
print("\n")

    Time : 50
    NumHits : 43/50
    Average minimum: 6560.205991622581




In [ ]:
print(suma)
print(counter)